In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA
from IPython.display import clear_output

In [ ]:
class MyML:
    def __init__(self):
        self.models = []
        self.is_normal = False
        self.predictions = []
        
        file_path = input('Please enter csv file path')
        self.load_data(file_path)
        
        self.add_models()
        self.call_pca_normalization()
        self.split_data()
        self.train()
        self._test()
               
    def load_data(self,file_name):
        self.data = pd.read_csv(file_name)
        
        while True:
            self.data.info()
            
            y_columns = input("Enter your y columns: ").replace(
                ' ','').split(",")
            
            if not set(y_columns).issubset(set(self.data.columns)):
                continue
            
            self.y = self.data[y_columns]
            
            break
        clear_output()
        
        while True:
            self.data.info()
            
            drop_columns = input("Enter Drop columns: ").replace(
                ' ','').split(",")
            print('drop_columns: ',drop_columns)
            if  drop_columns == ['']:
                break
            
            if not set(drop_columns).issubset(set(self.data.columns)):
                continue
            
            self.data = self.data.drop(drop_columns,axis=1)
            
            break
        clear_output()
        self.data.info()
        while True:
            self.data.info()
            
            dummies_columns = input("Enter Dummeis columns: ").replace(
                ' ','').split(",")
            
            if dummies_columns == ['']:
                break
            
            if not set(dummies_columns).issubset(set(self.data.columns)):
                continue
            
            for dum_col in dummies_columns:
                dummies_df = pd.get_dummies(self.data[dum_col],
                                            drop_first=True,dtype=int)
                
                self.data.drop(dum_col,axis=1,inplace=True)
                self.data = pd.concat([self.data,dummies_df],axis=1)
                
            
            
            break
            
        clear_output()
        self.x = self.data.drop(y_columns,axis=1)
        self.data.info()    
        
    
    def add_models(self):
        lookup_models = {'knn':{'requirement':{'k':None},
                                'SKmodel':KNeighborsClassifier}
                         ,'kmeans':{'requirement':{'k':None},
                                   'SKmodel':KMeans}
                         ,'logistic':{'SKmodel':LogisticRegression}
                         ,'dtree':{'SKmodel':DecisionTreeClassifier}
                         ,'RFC':{'requirement':{'n_estimators':None},
                                'SKmodel':RandomForestClassifier}
                         ,'svm':{'SKmodel':SVC}}
        
        model_name = ''
        
        while  model_name != 'ok':
            print(lookup_models.keys())
            
            model_name = input('your model? ')
            
            if not model_name in list(lookup_models.keys()):
                continue
            
            if 'requirement' in lookup_models[model_name]:
                args = []
                for key in lookup_models[model_name]['requirement']:
                    val  = int(input(f'value of {key} ? '))
                    lookup_models[model_name]['requirement'][key]=val
                    args.append(val)
                model = lookup_models[model_name]['SKmodel'](*args)
                
            else:
                model = lookup_models[model_name]['SKmodel']()
            
            self.models.append({model_name:model})
                
            
            
    
        
    def _normalization(self):
        # mode=['Standard','MinMax']
        mode = {"Standard":StandardScaler,'MinMax':MinMaxScaler}
        
        while True:
            answer = input("normalize?['Standard','MinMax','n']")
            print(answer)
            print(mode)
            if answer == "n":
                break
            elif answer in mode:
                scaler = mode[answer]()
                self.x = scaler.fit_transform(self.x)
                self.is_normal = True
            
                break
        
        
     
    def _pca(self):
        while True:
            try:
                n_component = int(input('n_component: / (0) == no pca'))
                if n_component == 0:
                    break
                pca = PCA(n_component)
                self.x = pca.fit_transform(self.x)
                break                
            except:
                pass
                
        
    def call_pca_normalization(self):
        self._normalization()
        if self.is_normal:
            self._pca()
        
    def split_data(self):
        while True:
            try:
                test_size = float(input('test_size?'))
                self.X_train, self.X_test, self.y_train,self.y_test = train_test_split(
                self.x,self.y, test_size=test_size,random_state=101)
                break
            except:
                pass
            
    
    
    def train(self):
        for d_model in self.models:
            try:
                model = list(d_model.values())[0]
                model.fit(self.X_train, self.y_train.values.ravel())
            except:
                pass              
        
    
    def _test(self):
        
        for d_model in self.models:
            try:
                model = list(d_model.values())[0]
                preds = model.predict(self.X_test)
                d = {list(d_model.keys())[0]:preds}
                self.predictions.append(d)
            except:
                pass    
    
    
    
    def result(self):
        acc = []
        model_name = []
        for preds in self.predictions:
            print("Model Name: " + list(preds.keys())[0])
            print(metrics.classification_report(self.y_test.to_numpy().ravel(),
                                         list(preds.values())[0]))
            p = metrics.classification_report(self.y_test.to_numpy().ravel(),
                                 list(preds.values())[0])
            print("*"*50)
                  
            report = metrics.classification_report(my_model.y_test.to_numpy().ravel(),
                                 list(preds.values())[0], output_dict  = True)
            acc.append(report["accuracy"])
            model_name.append(list(preds.keys())[0])
        
        sns.barplot(model_name, acc)
            
    
        

In [ ]:
my_model = MyML()

In [ ]:
my_model.result()